## Welcome to your notebook.


#### Run this cell to connect to your GIS and get started:

In [ ]:
action_sharepoint='Download_coleta'
action_sharepoint='Upload_items'

In [ ]:
pasta_sharepoint='Coleta'
delete_file=False

In [ ]:
pasta_sharepoint='Fundiario'
delete_file=False#True

In [ ]:
from arcgis.gis import GIS
gis = GIS("home")

#### Now you are ready to start!

In [ ]:
import csv
SECRET_CSV_ITEM_ID = ''
def get_secrets(gis=gis,
                secret_csv_item_id=SECRET_CSV_ITEM_ID):
    """Returns the secrets.csv file as a dict of secret_key : secret_value"""
    item = gis.content.get(secret_csv_item_id)
    with open(item.download(), 'r') as local_item_file:
        reader = csv.DictReader(local_item_file)
        return {rows['secret_key']: rows['secret_value'] for rows in reader}
secrets = get_secrets()

In [ ]:
import sys
sys.path.insert(0, "/arcgis/home/library")
from office365.runtime.auth.authentication_context import AuthenticationContext
from office365.runtime.auth.client_credential import ClientCredential
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.files.file import File
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.lists.list import List
import os
import time
sharepoint_user = 'alex.matias@pecenergia.com.br'
sharepoint_password=secrets["alex.matias@pecenergia.com.br"]
if(pasta_sharepoint=='Fundiario'):sharepoint_base_url = ''#site fundiario
elif(pasta_sharepoint=='Coleta'):sharepoint_base_url = ''#site fundiario
# Autenticar e obter a referência da lista de documentos
ctx = ClientContext(sharepoint_base_url).with_user_credentials(sharepoint_user, sharepoint_password)
auth = AuthenticationContext(sharepoint_base_url) 
auth.acquire_token_for_user(sharepoint_user, sharepoint_password)
ctx = ClientContext(sharepoint_base_url, auth)
web = ctx.web
ctx.load(web)
ctx.execute_query()
print('Conectado ao SharePoint: ',web.properties['Title'])

In [ ]:
import os

def folder_details(ctx, folder_in_sharepoint):  
    folder = ctx.web.get_folder_by_server_relative_url(folder_in_sharepoint)
    fold_names = []  
    sub_folders = folder.folders   
    ctx.load(sub_folders)  
    ctx.execute_query()
    #pega o nome dos arquivos dentro da pasta  
    for s_folder in sub_folders:
        fold_names.append(s_folder.properties["Name"]) 
    return fold_names

def files_in_folder(ctx,folder):
    folder = ctx.web.get_folder_by_server_relative_url(folder)  
    files = folder.files
    ctx.load(files)  
    ctx.execute_query()
    #pega o nome dos arquivos dentro da pasta  
    for file in files:
        if('.xlsm' in  file.properties["Name"] and len(file.properties["Name"])==8):return file.properties["Name"] 
    return ''
def return_pastas_coletas(ctx):
    pasta_coleta=''#site coleta
    folder_list = folder_details(ctx,pasta_coleta)
    return folder_list
if(action_sharepoint=='Download_coleta'):
    folder_list=return_pastas_coletas(ctx)
    folder_path='/arcgis/home/Coleta/'
    for folder in folder_list:
        file_name=files_in_folder(ctx,'' + '/' + folder)#site coleta
        if(file_name!=''):
            folder_GPD='/' + '/' + folder + '/' + file_name
            with open(os.path.join(folder_path,file_name), "wb") as local_file:
                file = ctx.web.get_file_by_server_relative_url(folder_GPD)
                file.download(local_file)
                ctx.execute_query()

In [ ]:
def upload_file(file_path, file_name, tipo,folder_list_coleta,pasta_sharepoint):
    folder_in_sharepoint=[]
    if(tipo=='Mod_GPD'):
        pasta=[item for item in folder_list_coleta if 'Mod' in item][0]
        if(pasta_sharepoint=='Fundiario'):folder_in_sharepoint=['']#pasta no fundiario
    elif(tipo=='Bases_kmz_atualizadas'):
        projeto=file_name[1:4].upper()
        if(pasta_sharepoint=='Fundiario'):folder_in_sharepoint=['']#pasta no fundiario
    elif(tipo=='BASES'):
        projeto=file_name[:3].upper()
        if(pasta_sharepoint=='Fundiario'):folder_in_sharepoint=['']#pasta no fundiario
    if(tipo=='Bases_kmz_atualizadas' or tipo=='BASES'):
        pasta=next((item for item in folder_list_coleta if item[:3].upper() == projeto), None)               
    if(pasta_sharepoint=='Coleta'):folder_in_sharepoint.append('/sites/GPD/FundirioComercial/'+pasta)
    
    # Ler o conteúdo do arquivo em um objeto bytes
    with open(os.path.join(file_path,file_name), 'rb') as f:
        file_content = f.read()
    name = os.path.basename(file_name)
    if(tipo=='Mod_GPD'):name = os.path.basename(time.strftime("%y_%m_%d")+'_'+file_name)
    for folder_share in folder_in_sharepoint:
        target_folder = ctx.web.get_folder_by_server_relative_url(folder_share)
        target_file = target_folder.upload_file(name, file_content).execute_query()
        print("File has been uploaded to url: {0}".format(target_file.serverRelativeUrl))

def delete_file(file_path, file_name, tipo):
    if(tipo=='BASES'):folder_in_sharepoint=['']#site no Fundiario e coletas
    else:folder_in_sharepoint = [''+tipo+'/']#site no Fundiario e coletas
    # Ler o conteúdo do arquivo em um objeto bytes
    for folder_share in folder_in_sharepoint:
        file_to_delete = ctx.web.get_folder_by_server_relative_url(folder_share+file_name)    
        file_to_delete.delete_object()
        ctx.execute_query()
        print("File has been deleted to url: {0}".format(file_to_delete.serverRelativeUrl))

if(action_sharepoint=='Upload_items'):
    caminho = '/arcgis/home'
    folder_list_coleta=return_pastas_coletas(ctx)
    for root, dirs, files in os.walk(caminho):
        for file in files:
            if('BASES'in root or 'Bases_kmz_atualizadas' in root or file=='modificacoes na base.xlsm'):
                if('Bases_kmz_atualizadas'in root):tipo='Bases_kmz_atualizadas'
                elif('BASES' in root):tipo='BASES'   
                #upload_file(root,file, tipo,folder_list_coleta)
                #if('BASES' not in root and file!='modificacoes na base.xlsm' ):delete_file(root, file, tipo)
                elif(file=='modificacoes na base.xlsm'):tipo='Mod_GPD'
                if(not(tipo=='Mod_GPD' and pasta_sharepoint=='Fundiario')):upload_file(root,file, tipo,folder_list_coleta,pasta_sharepoint)
                if(delete_file and file!='modificacoes na base.xlsm'):
                    print(os.path.join(root, file))
                    #os.remove(os.path.join(root, file))

#### Baixa os anexos para a tabela de solicitacoes

In [ ]:
Autores={97:'Bruno Sérgio Andrade Paiva',110:'Alex Matias',56:'Diógenes Baleeiro'}

def retorna_mes():
    from datetime import datetime
    currentMonth = datetime.now().month
    currentYear= datetime.now().year
    x=[]
    for i in range(1,12):x.append(i)
    y=['jan','fev','mar','abr','mai','jun','jul','ago','set','out','nov','dez']
    for i in range(len(x)):
        if(currentMonth==x[i]):mesAno=y[i]
    return mesAno

mes=retorna_mes()

In [ ]:
def retorna_df_areas_projetos(projetos):
    dict_layer = {}
    for projeto in projetos:
        map_projeto = gis.content.search('title:' + projeto, 'Feature Layer')
        Projeto_item = map_projeto[retorna_posicao(map_projeto, projeto)]
        Projeto_layers = Projeto_item.layers
        Projeto_tables = Projeto_item.tables
        dict_layer[projeto]={}
        for layer in Projeto_tables:
            lyr_fset = layer.query()
            lyr_sdf = lyr_fset.sdf
            dict_layer[projeto][layer.properties.name]={'sublayer': layer, 'sdf': lyr_sdf}
    return dict_layer


In [ ]:
def retorna_posicao(lista,elemento):
    """
    Retorna a posicao de um elemento em uma lista de objetos
    """
    for i in range(len(lista)):
        mapa=str(lista[i])
        split=mapa.replace('"','').replace(' type','').split(':')[1]
        if(split==elemento):
            return i 

In [ ]:
import shapefile
from shapely.geometry import shape

def filter_shapefile_latin_1(shapefile_path):
    reader = shapefile.Reader(shapefile_path, encoding='latin-1')

    filtered_records = []
    problematic_strings = []  # Lista para armazenar as strings problemáticas

    for shape_record in reader.shapeRecords():
        record = shape_record.record
        try:
            # Tenta criar a geometria a partir do shape_record
            geometry = shape(shape_record.shape.__geo_interface__)
            filtered_records.append((record, shape_record.shape))  # Armazena o registro e a geometria
        except UnicodeDecodeError:
            # Armazena a string problemática na lista
            problematic_strings.append(str(record))

    # Cria um novo shapefile somente com os registros filtrados
    writer = shapefile.Writer(shapefile_path[:-4])  # Remove a extensão .shp do caminho
    writer.fields = reader.fields[1:]  # Copia os campos do shapefile original, excluindo o campo de exclusão
    for record, geom in filtered_records:
        writer.record(*record)
        writer.shape(geom)

    # Fecha o objeto Writer e salva o novo shapefile
    writer.close()

    # Retorna o vetor com as strings problemáticas
    return problematic_strings

In [ ]:
import pandas as pd
from arcgis.geometry import Polygon, Geometry, Point, Polyline

def sdf_from_shp(shapefile_path):
    sdf = pd.DataFrame.spatial.from_featureclass(shapefile_path,encoding='iso-8859-1')
    return sdf

In [ ]:
def sharepoint_lists_fundiario(ctx,nome_lista):
        #verifica os itens nao incluidos na base que estao na lista
        """
        itens incluidos tem seu Title modificado para 'Incluido_na_base(editado_power_automate)' quando estao sem erro
        Quando estao com algum Erro de formato nao esperado de anexo , ou seja nao e um arquivo .zip com shapefiles dentro
        nomea-se "Erro1" e "Erro2" para erro nos campos do shapefile
        """
        titulos_esperados=['Incluido_na_base(editado_power_automate)','Erro1','Erro2']
        #Retorna a lista do sharepoint
        sp_site_lists = ctx.web.lists.get_by_title(nome_lista)
        all_items = sp_site_lists.items.paged(1000).get().execute_query()
        list_items = [item for item in all_items if item.properties["Title"] not in titulos_esperados]
        return list_items

In [ ]:
def erro1(list_item,erro):
    list_item.set_property('Title', "Erro1")
    dict_erros={1:'Erro de leitura de arquivo pode ser que nao e um arquivo .zip ou nao contem os arquivos necessarios para o .shp ou seja .shp, .shx e .dbf',
            2:'O pedido deve conter um anexo e apenas um anexo',
            3:'Shapefile tem de estar no padrao utf-8 , geralmente e causado por conter acentos ou cedilha nos campos de coluna ou campos de escrita ou esta fora do modelo'}
    string_erros=dict_erros[erro]
    list_item.set_property('Tipodeerro', string_erros)
    print(list_item)
    list_item.update()
    ctx.execute_query()

In [ ]:
import zipfile
def zipar_arquivos(nome_arquivo_zip,pasta):
    arquivos_zip = [arquivo for arquivo in os.listdir(pasta) if arquivo.endswith((".shp", ".shx", ".prj","dbf","cpg"))]
    
    with zipfile.ZipFile(os.path.join(pasta, nome_arquivo_zip), 'w', zipfile.ZIP_DEFLATED) as zipf:
        [zipf.write(os.path.join(pasta, arquivo), os.path.relpath(os.path.join(pasta, arquivo), pasta)) for arquivo in arquivos_zip]
    
    for arquivo in arquivos_zip:
        caminho_completo = os.path.join(pasta, arquivo)
        os.remove(caminho_completo)
    
    return os.path.join(pasta, nome_arquivo_zip)

In [ ]:
# CONSERTAR COM OS MOLDES CORRETOS    
def verifica_campos_shp(shapefile_path,tipo):
    #SO FUNCIONA PRO MOLDE SEM ACENTO
    print(shapefile_path)
    try :
        sdf = sdf_from_shp(shapefile_path)
    except:
        strings_com_erro=filter_shapefile_latin_1(shapefile_path)
        print(strings_com_erro)
        try: sdf = sdf_from_shp(shapefile_path)
        except:
            print("erro na codificacao , nao e latin-1")
            return False
    display(sdf)
    if(tipo=='Inclusao'):
        lista_campos=["Status","Proprieta","Matricula","Contrato","Imovel"]
        colunas=["Status","Proprieta"]
    if(tipo=='Edicao'):
        lista_campos=['area_code']
        colunas=['area_code']
    if(tipo=='Divisao'):
        lista_campos=['area_code',"Status","Proprieta","Matricula","Contrato","Imovel"]
        colunas=["Status","Proprieta"]
    
    if set(lista_campos).issubset(set(sdf.columns)):
        aceito=True
    else:
        aceito=False

    return aceito

In [ ]:
def cria_zip_legivel(roots, file_name):
    import zipfile
    import os
    import shutil
    # Define o nome do arquivo zip original e do arquivo zip novo
    file_name=file_name[:-4]
    # Percorre as subpastas da pasta extraída e move os arquivos para a pasta principal
    for root, dirs, files in os.walk('/arcgis/home/Nao Incluidos/temp'):
        for file in files:
            if(not(file.endswith('.zip'))):
                os.rename(os.path.join(root, file), os.path.join(root, file_name+file[-4:]))
                shutil.move(os.path.join(root, file_name+file[-4:]), os.path.join(roots, file_name+file[-4:]))
                
    caminho = '/arcgis/home/Nao Incluidos/temp'
    for root, dirs, files in os.walk(caminho):
            for dir in dirs:
                shutil.rmtree(os.path.join(root, dir))
def verificar_arquivo_shapefile_zip(nome_arquivo_zip):
    import zipfile

    # Lista de extensões esperadas
    extensoes_esperadas = ['shp', 'shx', 'dbf']  # Colocar extensao sem '.'

    # Dicionário para contar o número de ocorrências de cada extensão
    ocorrencias_extensoes = {extensao: 0 for extensao in extensoes_esperadas}

    # Função para verificar se um nome de arquivo corresponde a uma extensão esperada
    def verificar_extensao(nome_arquivo):
        extensao = nome_arquivo.split('.')[-1].lower()
        return extensao in extensoes_esperadas

    # Função para percorrer todas as pastas e subpastas dentro do arquivo zip
    def percorrer_zip(zip, caminho='', ocorrencias_extensoes=None):
        if ocorrencias_extensoes is None:
            ocorrencias_extensoes = {}
        for nome_arquivo in zip.namelist():
            partes_caminho = nome_arquivo.split('/')
            if nome_arquivo.startswith(caminho) and verificar_extensao(nome_arquivo):
                extensao = nome_arquivo.split('.')[-1].lower()
                ocorrencias_extensoes[extensao] = ocorrencias_extensoes.get(extensao, 0) + 1
        return ocorrencias_extensoes

    # Abrir o arquivo zip
    with zipfile.ZipFile(nome_arquivo_zip, 'r') as zip:
        # Percorrer todas as pastas e subpastas dentro do arquivo zip
        ocorrencias_extensoes = percorrer_zip(zip, ocorrencias_extensoes=ocorrencias_extensoes)
    
    print(ocorrencias_extensoes)

    # Verificar se cada extensão ocorreu apenas uma vez
    for extensao, ocorrencias in ocorrencias_extensoes.items():
        if ocorrencias != 1:
            return False

    return True

def error_report(arquivo_zip, tipo):
    if verificar_arquivo_shapefile_zip(arquivo_zip):
        print("É um arquivo zip")
        return True
    else:
        return False


In [ ]:
def download_attachments(ctx, list_item):
    # Obter anexos do item da lista
    attachments = list_item.attachment_files.get().execute_query()
    
    dict_item = {
            "Projeto": list_item.properties['Projeto'],
            "TipodeOperacaonabase": list_item.properties['TipodeOperacaonabase'],
            "ID_pedido": list_item.properties["Id"],
            "ID_author":list_item.properties["AuthorId"],
            "mes":mes,
            "erro":[]
            }

    
    #attachments = list_item.attachments
    if len(attachments) != 1:
        print("Arquivo .zip removidos do pedido de operacao na base de ID", list_item.properties["ID"], "esta incorreto") 
        dict_item["erro"] = 2
        return ('erro1', dict_item)

    attachment_title = attachments[0].properties["FileName"]
    if not attachment_title.endswith(".zip"):
        dict_item["erro"] = 1
        print("nao e um arquivo .zip")
        return ('erro1', dict_item)

    if list_item.properties["Attachments"]:
        for attachment in attachments:
            file_name = f"{dict_item['Projeto']}_{dict_item['TipodeOperacaonabase']}_{dict_item['ID_author']}_{dict_item['ID_pedido']}.zip"
            folder_path = f"/arcgis/home/Nao Incluidos/{dict_item['TipodeOperacaonabase']}"
            caminho = os.path.join(folder_path, file_name)

            with open(caminho, "wb") as f:
                content = attachment.download(f).execute_query()

            if verificar_arquivo_shapefile_zip(caminho):
                with zipfile.ZipFile(caminho) as zip_ref:    
                    zip_ref.extractall('/arcgis/home/Nao Incluidos/temp')    
                cria_zip_legivel(folder_path, file_name)
                print("Download de anexo com sucesso")
                caminho_shp = os.path.join(folder_path, file_name[:-4] + '.shp')
            else : 
                print("nao e arquivo .zip ou nao contem extensoes shapefile dentro")
                dict_item["erro"] = 1
                caminho_shp = 'erro1'

        os.remove(os.path.join(folder_path, file_name[:-4] + '.zip'))        
        return (caminho_shp, dict_item)

#### Adiciona arquivos na table do sharepoint , separando o erro 1 

In [ ]:
from arcgis.geometry import Polygon, Geometry, Point, Polyline
from arcgis.features import Feature,FeatureSet

def edit_features_sdf(sdf_edits,tipo,layer_projeto,Projeto,actions,path_file):
        """
            No caso das edicoes de Interseccoes e areas aprovadas podem ocorrer de ter mais de um caso para cada prop , 
            dessa forma para garantir que as edicoes nao deixem features "a mais" e necessario primeiro apagar e entao adicionar
        """ 
        if tipo in actions:
            sub_dicionario = actions[tipo]
            sublayers = list(sub_dicionario.keys())
        if tipo in actions:
            for sublayer in sublayers:
                if not sdf_edits[sublayer].empty:
                    #Monta o feature set para adicionar no arcgis
                    #print(layer_projeto[Projeto][sublayer]['sublayer'])
                    fields = [i['name'] for i in layer_projeto[Projeto][sublayer]['sublayer'].properties.fields if i['editable']]
                    if sublayer == 'Solicitacoes_Topografia' or sublayer =='Areas':
                        fields.append('SHAPE')
                    if tipo == 'Edicao':
                        fields.append('OBJECTID')
                    edit_fs_df = pd.DataFrame(columns=fields)
                    for col in edit_fs_df.columns:
                        if col in sdf_edits[sublayer]:
                            edit_fs_df[col] = sdf_edits[sublayer][col].values
                        df_dict = edit_fs_df.to_dict('records')
                        features = []
                        for record in df_dict:
                            feature = Feature(attributes=record)
                            features.append(feature)
                        feature_set = FeatureSet(features)
                        
                    #aplica as acoes necesserias 
                    action, arg = actions[tipo][sublayer]
                    if action == 'updates':
                        result = layer_projeto[Projeto][sublayer]['sublayer'].edit_features(updates=feature_set)
                    elif action == 'adds':
                        result = layer_projeto[Projeto][sublayer]['sublayer'].edit_features(adds=feature_set)

                    print(result)

                    if(result):
                        if(sublayer=='Sharepoint_files'):upload_attchment(layer_projeto[Projeto][sublayer]['sublayer'],path_file,result['addResults'][0]['objectId'])
                        if(tipo=='Edicao'):print("features editados com sucesso")
                        if(tipo=='Inclusao'):print("features incluidos com sucesso")

In [ ]:
import os 
import requests 
def upload_attchment(url,path_file,objectID):
    url.attachments.add(objectID, path_file)

In [ ]:
action_item_lista={'Inclusao': {'Sharepoint_files':('adds',False)}}
action_item_erro1={'Inclusao': {'Erro1':('adds',False)}}

In [ ]:
layer=retorna_df_areas_projetos(['Solicitacoes_topo'])

In [ ]:
def main():
    print('Conectado ao SharePoint: ',web.properties['Title'])
    items=sharepoint_lists_fundiario(ctx,'Solicitar Modificacao de base')
    exclusoes=pd.DataFrame(columns=["ID_pedido", "Projeto","Exclusao"])
    for item in items:
            print(item.properties["ID"])
            (caminho,data)=download_attachments(ctx,item)
            
            if(caminho!='erro1'):
                nome_arquivo = os.path.basename(caminho)
                diretorio = os.path.dirname(caminho)
                
                if(not(verifica_campos_shp(caminho,data["TipodeOperacaonabase"]))):
                    print("nao aceita")
                    data["erro"]=3
                    erro1(item,data["erro"])
                    df = pd.DataFrame([data])
                    edit_features_sdf({'Erro1':df},'Inclusao',layer,'Solicitacoes_topo',action_item_erro1,caminho)

                    
                    list_ext=['.dbf','.prj','.shp','.shx','cpg']
                    for ext in list_ext:
                        path = os.path.join(diretorio, nome_arquivo[:-4]+ext)
                        if (os.path.exists(path)):
                            os.remove(path)

                    print("Arquivo .zip removidos do pedido de operacao na base de ID",item.properties["ID"], "esta incorreto")            


                else:
                    df = pd.DataFrame([data])
                    caminho=zipar_arquivos(nome_arquivo[:-4]+'.zip',diretorio)
                    edit_features_sdf({'Sharepoint_files':df},'Inclusao',layer,'Solicitacoes_topo',action_item_lista,caminho)
                    os.remove(caminho)
                    print("Arquivo .zip :",os.path.basename(caminho),"do pedido de operacao na base de ID",item.properties["ID"], "esta correto") 

            else:
                df = pd.DataFrame([data])
                edit_features_sdf({'Erro1':df},'Inclusao',layer,'Solicitacoes_topo',action_item_erro1,caminho)
                erro1(item,data["erro"])

main()    

### Download attachments 

In [ ]:
caminho=layer['Solicitacoes_topo']['Sharepoint_files']['sublayer'].attachments.download(oid=6, attachment_id=1)[0]
with zipfile.ZipFile(caminho) as zip_ref:    
        zip_ref.extractall('/arcgis/home/Nao Incluidos/temp')
        nome_arquivo = os.path.basename(caminho)
        diretorio = '/arcgis/home/Nao Incluidos/temp'

        caminho_shp = os.path.join(diretorio, nome_arquivo[:-4] + '.shp')
        sdf=sdf_from_shp(caminho_shp)
        display(sdf)
        ########### CODIGO DE VERIFICACAO DE BASE
        os.remove(caminho_shp)